In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Paramètre à modifier : budget total (en M£, ou multiplie x10 si prix en 0.1M£)
my_budget = 1500

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Pondérations pour chaque critère (à ajuster selon l’importance donnée à chaque indicateur)
w_points_per_cost = 0.2  # positif : favorise joueurs rapportant beaucoup de points par rapport à leur coût
w_ict_index = 0.1
w_minutes_last = 0.1 # positif : favorise joueurs ayant joué récemment
w_total_points_last = 0.2 # positif : favorise joueurs en forme
w_ict_last = 0.2   # positif : favorise joueurs en forme
w_xGI_last = 0.1  # Poids ajouté sur xG_recent pour MID et FWD
w_goals_conceded_per_90 = -0.2 # négatif : pénalise DEF et GKP concédant beaucoup de buts
w_saves_per_90 = 0.1 # Poids ajouté sur saves pour GKP
w_clean_sheets_per_90 = 0.1 # Poids ajouté sur clean_sheets_per_90 pour DEF et GKP
w_defensive_contribution_per_90 = 0.1 # Poids ajouté sur defensive_contribution_per_90 pour DEF et GKP
w_fdr = -0.3   # négatif : pénalise calendrier difficile
w_total_points_last_per_xGI_last = -0.0 # négatif : met en avant les joueurs sous-côtés

# Variable d’état pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : score multi-indicateurs pondéré
prob = LpProblem("FPL_Optimizer", LpMaximize)
prob += lpSum([
    w_points_per_cost  * df_players.loc[i, 'points_per_cost'] * player_vars[i]
  + w_ict_index       * df_players.loc[i, 'ict_index'] * player_vars[i]
  + w_minutes_last * df_players.loc[i, 'minutes_last'] * player_vars[i]
  + w_total_points_last * df_players.loc[i, 'total_points_last'] * player_vars[i]
  + w_ict_last       * df_players.loc[i, 'ict_last'] * player_vars[i]
  + (w_goals_conceded_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['goals_conceded_per_90'] * player_vars[i]
  + (w_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.iloc[i]['xGI_last'] * player_vars[i]
  + (w_saves_per_90 if df_players.iloc[i]['position'] in ['GKP'] else 0) * df_players.iloc[i]['saves_per_90'] * player_vars[i]
  + (w_clean_sheets_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['clean_sheets_per_90'] * player_vars[i]
  + (w_defensive_contribution_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['defensive_contribution_per_90'] * player_vars[i]
  + w_fdr             * df_players.loc[i, 'fdr_next_6'] * player_vars[i]
  + (w_total_points_last_per_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.loc[i, 'total_points_last_per_xGI_last'] * player_vars[i]
    for i in range(len(df_players))
    if player_vars[i] == 1
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte de budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= my_budget

# Contrainte max 3 joueurs de la même équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i] for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Résolution
prob.solve()

# Résultats : joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/b9f96e8262344b04be09d495f81e2cc7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b9f96e8262344b04be09d495f81e2cc7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 5226 RHS
At line 5253 BOUNDS
At line 5996 ENDATA
Problem MODEL has 26 rows, 742 columns and 2968 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 184.404 - 0.00 seconds
Cgl0004I processed model has 26 rows, 626 columns (626 integer (554 of which binary)) and 2504 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -184.404
Cbc0038I Cleaned solution of -184.404
Cbc0038I Before mini branch and bound, 626 integers at bound fixed and 0 continuous
Cbc003

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,clean_sheets_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Omar,Alderete,SUN,Sunderland,DEF,41.0,39.0,0.951220,0.080082,3.000000,29.8,6.1,63,7.5,487.0,673728,2200,83384,53,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.22,0.74,10.90,0.37,67.50,0.1625,0.1175,4.625,7.50,46.153846
1,Marcos,Senesi,BOU,Bournemouth,DEF,49.0,45.0,0.918367,0.078809,3.166667,28.1,24.0,14,5.0,571.0,2853986,3437,196153,89,2.0,0.0,2.0,0.14,0.41,0.55,3.636364,0.02,0.00,0.06,0.08,0.87,1.10,13.08,0.47,76.00,0.0800,0.0000,3.900,5.00,62.500000
8,Tyrick,Mitchell,CRY,Crystal Palace,DEF,50.0,34.0,0.680000,0.062963,3.166667,32.7,2.5,117,4.5,540.0,202528,348,87001,41,0.0,1.0,1.0,0.28,0.46,0.74,1.351351,0.05,0.00,0.08,0.13,1.14,0.50,8.50,0.50,67.50,0.1575,0.0700,5.150,4.50,28.571429
15,Jurriën,J.Timber,ARS,Arsenal,DEF,58.0,37.0,0.637931,0.085847,2.500000,43.2,15.7,21,2.8,431.0,1730989,1599,399897,312,1.0,2.0,3.0,1.73,0.38,2.11,1.421801,0.36,0.00,0.08,0.44,0.59,0.63,7.52,0.42,64.75,0.2775,0.2075,4.875,2.75,9.909910
25,Gabriel,Gabriel,ARS,Arsenal,DEF,62.0,38.0,0.612903,0.070370,2.500000,24.1,27.9,11,6.0,540.0,1751602,1355,699641,2809,0.0,1.0,1.0,0.48,0.06,0.54,1.851852,0.08,0.00,0.01,0.09,0.65,0.50,9.00,0.50,67.50,0.1150,0.1050,4.300,6.00,52.173913
51,João Pedro,João Pedro,CHE,Chelsea,FWD,77.0,37.0,0.480519,0.072549,2.666667,40.2,62.7,1,2.8,510.0,2758918,866,1755992,1935,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,0.35,64.50,0.0575,0.0375,3.000,2.75,47.826087
67,Erling,Haaland,MCI,Man City,FWD,144.0,62.0,0.430556,0.123260,3.333333,70.6,57.9,2,9.5,503.0,4260583,3653,619137,45,1.0,8.0,9.0,7.35,0.56,7.91,1.137800,1.32,0.00,0.10,1.42,1.14,0.89,4.29,0.54,62.75,0.9375,0.8725,10.450,9.50,10.133333
68,Jarrod,Bowen,WHU,West Ham,FWD,77.0,33.0,0.428571,0.061111,2.833333,41.7,6.4,60,5.0,540.0,260452,1060,1310081,218,0.0,3.0,3.0,0.83,0.50,1.33,2.255639,0.14,0.00,0.08,0.22,1.41,2.33,5.67,0.17,67.50,0.2050,0.1300,5.350,5.00,24.390244
3,Robin,Roefs,SUN,Sunderland,GKP,46.0,39.0,0.847826,0.072222,3.000000,19.9,4.3,84,5.5,540.0,508144,901,103453,36,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,0.50,67.50,0.0025,0.0000,2.950,5.50,2200.000000
33,Đorđe,Petrović,BOU,Bournemouth,GKP,45.0,25.0,0.555556,0.043783,3.166667,13.6,5.7,65,3.0,571.0,504856,308,214224,122,0.0,0.0,0.0,0.00,0.00,0.00,NaN,0.00,2.52,0.00,0.00,0.87,1.10,0.00,0.47,75.75,0.0000,0.0000,1.650,3.00,0.000000


In [4]:
selected_df['price'].sum()

np.float64(1010.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(348.59999999999997)